In [0]:
pip install geopy

In [0]:
import csv
import collections
import math
import openlocationcode as olc
from geopy import distance
# Plus Code Precision Level
# 10 => 14m
# 8 => 275m
# 6 => 5.55KM
# 4 => 111KM
# 2 => 2220KM

In [0]:
import datetime
def format_date(date):
  return datetime.datetime.strptime(date, "%m/%d/%Y")

In [0]:
PLUS_CODE_PRECISION = 4
predicted_data = {}
pred = open("predicted_data.csv")
reader1 = csv.DictReader(pred)
for row in reader1:
  date = format_date(row['Date'])
  if date in predicted_data:
    row['plus_code'] = olc.encode(float(row['lat']), float(row['long']), PLUS_CODE_PRECISION)[0:4]
    predicted_data[date].append(dict(row))
    predicted_data[date] = sorted(predicted_data[date], key= lambda x: x['plus_code'])
  else:
    row['plus_code'] = olc.encode(float(row['lat']), float(row['long']), PLUS_CODE_PRECISION)[0:4]
    predicted_data[date] = [dict(row)]

predicted_dates = sorted(predicted_data.keys())

In [0]:
actual_data = {}
actual = open("actual_data.csv")
reader1 = csv.DictReader(actual)
for row in reader1:
  date = format_date(row['date'])

  if date in actual_data:
    row['plus_code'] = olc.encode(float(row['latitude']), float(row['longitude']), PLUS_CODE_PRECISION)[0:4]
    actual_data[date].append(dict(row))
    actual_data[date] = sorted(actual_data[date], key=lambda x: x['plus_code'])
  else:
    row['plus_code'] = olc.encode(float(row['latitude']), float(row['longitude']), PLUS_CODE_PRECISION)[0:4]
    actual_data[date] = [dict(row)]

actual_dates = sorted(actual_data.keys())

In [0]:
def search_nearest_to_actual(date, left = 0, right = len(actual_dates)):
  if right >= left:
    mid = int((left + right) / 2)
    if(actual_dates[mid] == date):
      return ["exact_found", mid]
    else:
      if date > actual_dates[mid]:
        return search_nearest_to_actual(date, mid + 1, right)
      else:
        return search_nearest_to_actual(date, left, mid - 1)
  else:
    return ["nearest_found", left]

In [0]:
def day_score(diff):
  if diff == 0:
    return 1
  elif diff == 1:
    return 0.75
  elif diff == 2:
    return 0.5
  elif diff == 3:
    return 0.25
  else:
    return 0

def distance_score(dist):
  if dist > 400:
    return 0
  elif dist > 300:
    return 0.25
  elif dist > 200:
    return 0.5
  elif dist > 100:
    return 0.25
  else:
    return 1

max_threshold_distance = 400
max_threshold_days = 4

In [0]:
for date in predicted_dates:
  status, index = search_nearest_to_actual(date)
  next_actual_date = actual_dates[index]
  current_diff = abs((date - next_actual_date).days)

  if current_diff > 4:
    predicted_data[date]['score'] = 0
    continue
  predicted_event_count = len(predicted_data[date])
  for predict_row in predicted_data[date]:
    max_score = 0
    first = (predict_row['lat'], predict_row['long'])
    actual_event_count = len(actual_data[next_actual_date])
    for actual_row in actual_data[next_actual_date]:
      try:
        second = (float(actual_row['latitude']), float(actual_row['longitude']))	
        current_dist = distance.distance(first, second).km
      except ValueError as e:
        print("wrong lat lon", actual_row, predict_row)
        continue
      score = day_score(current_diff) * distance_score(current_dist)
      ratio = float(min(predicted_event_count,actual_event_count))  / float(max(predicted_event_count,actual_event_count))
      final_score = 0.2 * ratio + 0.8 * score
      if final_score > max_score:
        max_score = final_score
        predict_row['score'] = final_score
    if index < len(actual_dates):
      next_actual_date = actual_dates[index]
      current_diff = abs((date - next_actual_date).days)
      index += 1
    if current_diff > 4:
      break

print(predicted_data)

In [0]:
final_result = open("final_result.csv", "w")
final_result.write("country,date,lat,long,place,state,score")
final_result.write("\r\n")
for date in predicted_data.keys():
  string_date = datetime.datetime.strftime(date, "%m/%d/%Y")
  for row in predicted_data[date]:
    final_result.write("{0},{1},{2},{3},{4},{5},{6}".format(row['country'], string_date, row['lat'], row['long'], row['place'], row['state'], row['score'] if 'score' in row else 0))
    final_result.write("\r\n")
final_result.close()